**LIKENESS SCORE**

In [1]:
import glob
import numpy as np
import scipy.misc
from scipy.spatial.distance import minkowski
from scipy.stats import ks_2samp
import time,imageio, os
import torch
from PIL import Image
from tqdm import tqdm
import cv2

from google.colab import drive, auth
import gspread
from google.auth import default

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def get_gen_image_vector(filename):

    im = Image.open(filename)
    resized_image = im.resize((128, 128))
    image_array = np.array(resized_image)
    print(image_array.shape)

    return np.float32(np.ndarray.flatten(image_array))/255

In [4]:
def get_real_image_vector(filename):

    im = Image.open(filename)
    resized_image = im.resize((128, 128))
    image_array = np.array(resized_image)
    gray_image = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    print(gray_image.shape)

    return np.float32(np.ndarray.flatten(gray_image))/255

In [5]:
def dists(data):
    num = data.shape[0]
    data = data.reshape((num, -1))
    dist = []
    for i in tqdm(range(0,num-1)):
        for j in range(i+1,num):
            dist.append(minkowski(data[i],data[j]))

    return np.array(dist)

def dist_btw(a,b):
    a = a.reshape((a.shape[0], -1))
    b = b.reshape((b.shape[0], -1))
    dist = []
    for i in tqdm(range(a.shape[0])):
        for j in range(b.shape[0]):
            dist.append(minkowski(a[i],b[j]))

    return np.array(dist)


def LS(real,gen):
    dist_real = dists(real)
    dist_gen = dists(gen)
    distbtw = dist_btw(real, gen)
    D_Sep_1, _ = ks_2samp(dist_real, distbtw)
    D_Sep_2, _ = ks_2samp(dist_gen, distbtw)

    return 1- np.max([D_Sep_1, D_Sep_2])

In [8]:
# Path to Generation Dataset VAE
%cd /content/drive/MyDrive/BrainTumorDataSet/generated_images_vae
generated_files_vae = glob.glob('*.png')
generated_files_vae = np.array([get_gen_image_vector(filename) for filename in generated_files_vae])

# Path to Generation Dataset GAN
%cd /content/drive/MyDrive/BrainTumorDataSet/generated_images_gan
generated_files_gan = glob.glob('*.png')
generated_files_gan = np.array([get_gen_image_vector(filename) for filename in generated_files_gan])

# Path to Real Dataset
%cd /content/drive/MyDrive/BrainTumorDataSet/Real
real_files = glob.glob('*.jpg')

real = np.array([get_real_image_vector(filename) for filename in real_files])

print('No.of Original Images :   '+str(len(real)))
print('No.of Generated VAE Images:   '+str(len(generated_files_vae)))
print('No.of Generated GAN Images:   '+str(len(generated_files_vae)))

print('\n', 'Likeness Score (Real, VAE)= ', LS(real, generated_files_vae))
print('\n', 'Likeness Score (Real, GAN)= ', LS(real, generated_files_gan))

/content/drive/.shortcut-targets-by-id/1ZFHcykFy2C0_n5ntfNGkr8VRWvtSqULv/BrainTumorDataSet/generated_images_vae
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
/content/drive/.shortcut-targets-by-id/1ZFHcykFy2C0_n5ntfNGkr8VRWvtSqULv/BrainTumorDataSet/generated_images_gan
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
/content/drive/.shortcut-targets-by-id/1ZFHcykFy2C0_n5ntfNGkr8VRWvtSqULv/BrainTumorDataSet/Real
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
No.of Original Images :   10
No.of Generated VAE Images:   10
No.of Generated GAN Images:   10


100%|██████████| 10/10 [00:00<00:00, 1349.56it/s]



 Likeness Score (Real, VAE)=  0.7433333333333334


100%|██████████| 10/10 [00:00<00:00, 1235.36it/s]


 Likeness Score (Real, GAN)=  0.0
